## Projeto Twitter Febre Amarela

### Importa todos os pacotes de Processamento de Linguagem Natural

In [114]:
import nltk
#nltk.download('all')

In [115]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict


**Ler arquivo de dados e conta a quantidade de linhas**

In [368]:
# Dados coletados e Classificados pelo Data Scientist Rodrigo Santana Ferreira 
dataset = pd.read_excel('dados/classificacao_febre_amarela2.xlsx',encoding='utf-8')

In [369]:
# número de tuplas
print("Número de linhas: ",len(dataset))

Número de linhas:  4009


In [370]:
# número de atributos
print("Número de atributos: ",len(dataset.columns))

Número de atributos:  8


In [371]:
# atributos
dataset.columns

Index(['city', 'data', 'id_str', 'lang', 'retweeted', 'state', 'classificação',
       'text'],
      dtype='object')

In [372]:
# tweets Já classificados - Positivos, Negativos e Neutros
dataset.head(8)

city                data              id_str lang  retweeted state  \
0  null 2018-04-11 03:01:44  983902892909387776   pt          0  null   
1  null 2018-04-11 02:56:35  983901596311605248   pt          0  null   
2  null 2018-04-11 02:55:31  983901330220683264   pt          0  null   
3  null 2018-04-11 02:46:41  983899105637388289   pt          0  null   
4  null 2018-04-11 02:38:29  983897040856940545   pt          0  null   
5  null 2018-04-11 02:22:58  983893135787986945   pt          0  null   
6  null 2018-04-11 02:21:32  983892778840219648   pt          0  null   
7  null 2018-04-11 01:58:47  983887050800861184   pt          0  null   

  classificação                                               text  
0       notícia  Novo teste identifica se infecção de febre ama...  
1       notícia  #OMS protegerá um bilhão de pessoas na #África...  
2       notícia  Pulseira Repelente Espanta Pernilongo,Mosquito...  
3        doença  Fui ver meu amigo que tava com febre amarela e...  
4       notícia  Macacos NÃO transmitem a febre amarela!!!!!! r...  
5        outros  Nego gosta de avacalhar né, pra que colocar gl...  
6       notícia  O problema de aparecer algo estranho no seu co...  
7        outros      @eaisandyyy Tô matando mais que febre amarela

In [373]:
# eliminando alguns atributos do dataset
#atributos = ['Text', 'Classificacao', 'Created At', 'Geo Coordinates.latitude', 'Geo Coordinates.longitude',
#             'User Location', 'User Screen Name',  ]
#dataset2 = dataset.loc[:,atributos]
#dataset = dataset2
#dataset.head(5)

In [374]:
type(dataset)

pandas.core.frame.DataFrame

In [375]:
dataset.tail()

city                data               id_str lang  retweeted state  \
4004  null 2018-05-25 22:48:35  1000146638915211264   pt          0  null   
4005  null 2018-05-25 22:40:18  1000144555176595458   pt          0  null   
4006  null 2018-05-25 22:28:33  1000141599412760576   pt          0  null   
4007  null 2018-05-25 22:28:09  1000141498997002241   pt          0  null   
4008  null 2018-05-25 20:39:44  1000114212524494848   pt          0  null   

     classificação                                               text  
4004        outros  #SP2 a gente naum sabe se chega em casa por ca...  
4005        outros  O lado bom dessas greve dos caminhoneiros é qu...  
4006        outros  @tavares_gremio @MichelTemer toda vez que ele ...  
4007        outros  A fila dos postos de gasolina ta parecendo a f...  
4008        outros  e eu passando vergonha com vacina da febre ama...

In [376]:
dataset.classificação.value_counts()

outros       1558
notícia      1047
prevenção     749
doença        649
Name: classificação, dtype: int64

**Conta a quantidade de linhas de tweets neutros, positivos e negativos**

In [377]:
len(dataset[dataset.classificação=='outros'])

1558

In [378]:
len(dataset[dataset.classificação=='notícia'])

1047

In [379]:
len(dataset[dataset.classificação=='prevenção'])

749

## Pre-Processamento 

** Separando tweets e suas classes
**

In [380]:
tweets = dataset['text'].values
classes = dataset['classificação'].values

In [381]:
import re
document = 'rt @prettycolleges: university of phoenix http://t.co/d5wxsy332r good'

print (re.sub(r'http:\\*/\\*/.*?\s', ' ', document)) ## note the r (raw string)

rt @prettycolleges: university of phoenix  good


In [382]:
re.sub(r'http\S+', '', document)

'rt @prettycolleges: university of phoenix  good'

In [383]:
import re
tweets2 = tweets.copy()
tweets3 = re.sub(r'http\S+', '', tweets2[0]) ## note the r (raw string)
tweets3

'Novo teste identifica se infecção de febre amarela foi por vírus ou vacina'

In [384]:
type(tweets[0])

str

In [385]:
# elimina url
def elimina_url(texto):
    #print(type(texto))
    texto = str(texto)
    return re.sub(r'http\S+', '', texto)

tweets4 = tweets.copy()
for i in range(len(tweets)):
    tweets4[i] = elimina_url(tweets4[i])
    
tweets4[:10]

array([ 'Novo teste identifica se infecção de febre amarela foi por vírus ou vacina',
       '#OMS protegerá um bilhão de pessoas na #África contra a #febreamarela até 2026 Fonte: ',
       'Pulseira Repelente Espanta Pernilongo,Mosquito da Dengue,Febre Amarela Proteja toda Familia ! Entregamos no metro de são paulo consulte !',
       'Fui ver meu amigo que tava com febre amarela e mano ele superou metade já! Graças a Deus deu tudo certo',
       'Macacos NÃO transmitem a febre amarela!!!!!! repassa aí galera, qualquer tipo de ajuda é sempre bem vinda nessas situações',
       'Nego gosta de avacalhar né, pra que colocar glitter em um trabalho sobre febre amarela?????',
       'O problema de aparecer algo estranho no seu corpo é que o google sempre exagera, Já peguei febre amarela por ela e agora estou me cagando de medo com medo de ter câncer. É isso que o sensacionalismo faz… ',
       '@eaisandyyy Tô matando mais que febre amarela',
       'Quase todo mundo acha q a vacina da febre

In [386]:
# colocar no dataframe
dataset2 = dataset.copy()
dataset2.Text = tweets4
dataset2.head()

city                data              id_str lang  retweeted state  \
0  null 2018-04-11 03:01:44  983902892909387776   pt          0  null   
1  null 2018-04-11 02:56:35  983901596311605248   pt          0  null   
2  null 2018-04-11 02:55:31  983901330220683264   pt          0  null   
3  null 2018-04-11 02:46:41  983899105637388289   pt          0  null   
4  null 2018-04-11 02:38:29  983897040856940545   pt          0  null   

  classificação                                               text  
0       notícia  Novo teste identifica se infecção de febre ama...  
1       notícia  #OMS protegerá um bilhão de pessoas na #África...  
2       notícia  Pulseira Repelente Espanta Pernilongo,Mosquito...  
3        doença  Fui ver meu amigo que tava com febre amarela e...  
4       notícia  Macacos NÃO transmitem a febre amarela!!!!!! r...

In [387]:
# salvar para excel
#dataset2.to_excel('classificacao-febreamarela-2018.xlsx')

In [388]:
tweets4[:10]

array([ 'Novo teste identifica se infecção de febre amarela foi por vírus ou vacina',
       '#OMS protegerá um bilhão de pessoas na #África contra a #febreamarela até 2026 Fonte: ',
       'Pulseira Repelente Espanta Pernilongo,Mosquito da Dengue,Febre Amarela Proteja toda Familia ! Entregamos no metro de são paulo consulte !',
       'Fui ver meu amigo que tava com febre amarela e mano ele superou metade já! Graças a Deus deu tudo certo',
       'Macacos NÃO transmitem a febre amarela!!!!!! repassa aí galera, qualquer tipo de ajuda é sempre bem vinda nessas situações',
       'Nego gosta de avacalhar né, pra que colocar glitter em um trabalho sobre febre amarela?????',
       'O problema de aparecer algo estranho no seu corpo é que o google sempre exagera, Já peguei febre amarela por ela e agora estou me cagando de medo com medo de ter câncer. É isso que o sensacionalismo faz… ',
       '@eaisandyyy Tô matando mais que febre amarela',
       'Quase todo mundo acha q a vacina da febre

In [389]:
classes[:10]

array(['notícia', 'notícia', 'notícia', 'doença', 'notícia', 'outros',
       'notícia', 'outros', 'notícia', 'prevenção'], dtype=object)

**Gerando o modelo**

In [390]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets4)
modelo = MultinomialNB()
modelo.fit(freq_tweets, list(classes))
#list(freq_tweets.todense()[0])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [391]:
type(classes)

numpy.ndarray

** Testando o modelo com algumas instâncias simples**

In [392]:
testes = ['Novo teste identifica se infecção de febre amarela foi por vírus ou vacina ',
       'Mulheres com endometriose podem tomar a vacina da febre amarela? \n ',
       '#EusouFera: Antes de jogo contra o Palmeiras, Boca Juniors toma vacina da febre amarela -  ',
       '#OMS protegerá um bilhão de pessoas na #África contra a #febreamarela até 2026 Fonte: ',
       'Pulseira Repelente Espanta Pernilongo,Mosquito da Dengue,Febre Amarela Proteja toda Familia ! Entregamos no metro de são paulo consulte !',
       'Fui ver meu amigo que tava com febre amarela e mano ele superou metade já! Graças a Deus deu tudo certo',
       'Macacos NÃO transmitem a febre amarela!!!!!! repassa aí galera, qualquer tipo de ajuda é sempre bem vinda nessas situações',
       'Nego gosta de avacalhar né, pra que colocar glitter em um trabalho sobre febre amarela?????',
       'O problema de aparecer algo estranho no seu corpo é que o google sempre exagera, Já peguei febre amarela por ela e agora estou me cagando de medo com medo de ter câncer. É isso que o sensacionalismo faz… ',
       '@irhartje @PHGoshiki Febre amarela, ctz']

In [393]:
freq_testes = vectorizer.transform(testes)

In [394]:
freq_testes.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [395]:
# Fazendo a classificação com o modelo treinado.
modelo.predict(freq_testes)

array(['notícia', 'prevenção', 'prevenção', 'notícia', 'prevenção',
       'outros', 'notícia', 'outros', 'outros', 'doença'],
      dtype='<U9')

In [396]:
tweets = dataset2['text']
classes = dataset2['classificação']

In [397]:
type(tweets)

pandas.core.series.Series

In [398]:
df = pd.DataFrame()
df['tweet'] = testes
df['classificacao'] = modelo.predict(freq_testes)

tweets = testes
classificacao = modelo.predict(freq_testes)

print ("Novas Instancias Classificadas...")
for i in range(len(df)):
    print (classificacao[i], " -> ", tweets[i] )

Novas Instancias Classificadas...
notícia  ->  Novo teste identifica se infecção de febre amarela foi por vírus ou vacina 
prevenção  ->  Mulheres com endometriose podem tomar a vacina da febre amarela? 
 
prevenção  ->  #EusouFera: Antes de jogo contra o Palmeiras, Boca Juniors toma vacina da febre amarela -  
notícia  ->  #OMS protegerá um bilhão de pessoas na #África contra a #febreamarela até 2026 Fonte: 
prevenção  ->  Pulseira Repelente Espanta Pernilongo,Mosquito da Dengue,Febre Amarela Proteja toda Familia ! Entregamos no metro de são paulo consulte !
outros  ->  Fui ver meu amigo que tava com febre amarela e mano ele superou metade já! Graças a Deus deu tudo certo
notícia  ->  Macacos NÃO transmitem a febre amarela!!!!!! repassa aí galera, qualquer tipo de ajuda é sempre bem vinda nessas situações
outros  ->  Nego gosta de avacalhar né, pra que colocar glitter em um trabalho sobre febre amarela?????
outros  ->  O problema de aparecer algo estranho no seu corpo é que o google s

** Avaliando o modelo **

In [399]:
# Fazendo o cross validation do modelo
resultados = cross_val_predict(modelo, freq_tweets, list(classes), cv=5)
resultados

array(['notícia', 'notícia', 'prevenção', ..., 'outros', 'outros', 'outros'],
      dtype='<U9')

In [400]:
resultados[:10]

array(['notícia', 'notícia', 'prevenção', 'outros', 'outros', 'outros',
       'outros', 'outros', 'outros', 'outros'],
      dtype='<U9')

In [401]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(list(classes),resultados)

0.69393863806435518

### Matriz de Confusão

In [402]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['REAL'], colnames=['PREDITO'], margins=True))

PREDITO    doença  nan  notícia  outros  prevenção   All
REAL                                                    
doença        476    0       33     128         12   649
notícia        55    0      722     214         56  1047
outros         37    3      140    1212        166  1558
prevenção       2    0      131     244        372   749
All           570    3     1026    1798        606  4003


In [403]:
# Medidas de validação do modelo
sentimento=['doença',  'nan',  'notícia',  'outros',  'prevenção' ]
print (metrics.classification_report(list(classes),resultados,sentimento))

             precision    recall  f1-score   support

     doença       0.83      0.73      0.78       649
        nan       0.00      0.00      0.00         6
    notícia       0.70      0.69      0.70      1047
     outros       0.67      0.78      0.72      1558
  prevenção       0.61      0.50      0.55       749

avg / total       0.69      0.69      0.69      4009



In [404]:
modelo

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# gerar o modelo em arquivo

In [426]:
# gerar o modelo
import pickle

file = "modelo.dat"

with open(file, "wb") as f:
    pickle.dump(modelo, f)
    print('modelo salvo...')

modelo salvo...


In [427]:
!ls -hla

total 3,3M
drwxrwxr-x  4 eleonilia eleonilia 4,0K out 30 22:12 .
drwxr-xr-x 14 eleonilia eleonilia 4,0K jul 12 12:04 ..
-rw-r--r--  1 eleonilia eleonilia  51K out 29 23:52 AnaliseDeSentimentos-GovernoMinasGerais.ipynb
-rw-rw-r--  1 eleonilia eleonilia  38K mar  8  2018 AnaliseDeSentimentos-SearchTweets-Versao-3.0.ipynb
-rwxrwxrwx  1 eleonilia eleonilia 382K out 30 22:09 classificacao_febre_amarela2.xlsx
drwxrwxr-x  2 eleonilia eleonilia 4,0K out 30 22:12 Dados Febre Amarela
drwxr-xr-x  2 eleonilia eleonilia 4,0K out 29 17:38 .ipynb_checkpoints
-rw-r--r--  1 eleonilia eleonilia 620K out 30 22:13 modelo.dat
-rw-r--r--  1 eleonilia eleonilia 275K jul 10 18:11 mozilla.pdf
-rw-rw-r--  1 eleonilia eleonilia 246K out 30 22:11 Projeto_Twiiter-FebreAmarela-Principal.ipynb
-rw-rw-r--  1 eleonilia eleonilia 199K mar  7  2018 Twitter_demo.ipynb
-rw-rw-r--  1 eleonilia eleonilia 1,4M jul 29 03:32 twitter-epidemiologia-sarampo.ipynb
-rw-r--r--  1 eleonilia eleonilia  80K jul 10 12:20 twitter_febre_a

In [428]:
with open(file, "rb") as f:
    m = pickle.load(f)
    print('modelo carregado...')
    
    
vectorizer = CountVectorizer(analyzer="word")

freq_tweets = vectorizer.fit_transform(tweets4)
modelo = MultinomialNB()
modelo.fit(freq_tweets, list(classes))    
    
    
resultados = cross_val_predict(m, freq_tweets, list(classes), cv=10)
# Medindo a acurácia média do modelo
print(metrics.accuracy_score(list(classes),resultados))
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['REAL'], colnames=['PREDITO'], margins=True))

modelo carregado...
0.693689199302
PREDITO    doença  nan  notícia  outros  prevenção   All
REAL                                                    
doença        480    0       33     124         12   649
notícia        59    0      729     208         51  1047
outros         37    3      143    1183        192  1558
prevenção       2    0      130     228        389   749
All           578    3     1035    1743        644  4003


/home/eleonilia/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


# Classificar os tweets com coordenadas

In [429]:
!ls

AnaliseDeSentimentos-GovernoMinasGerais.ipynb
AnaliseDeSentimentos-SearchTweets-Versao-3.0.ipynb
classificacao_febre_amarela2.xlsx
Dados Febre Amarela
modelo.dat
mozilla.pdf
Projeto_Twiiter-FebreAmarela-Principal.ipynb
Twitter_demo.ipynb
twitter-epidemiologia-sarampo.ipynb
twitter_febre_amarela.xlsx


In [430]:
dfcoord = pd.read_excel('twitter_febre_amarela.xlsx', encoding='utf-8')
dfcoord.head()

city                data              id_str lang retweeted  \
0     Nilopolis 2018-03-13 20:27:36  973656844601315328   pt     false   
1  Praia Grande 2018-03-13 19:57:57  973649386491260929   pt     false   
2     Guaratuba 2018-03-13 19:53:57  973648377043922947   pt     false   
3       Itaguai 2018-03-13 19:16:13  973638884155326464   pt     false   
4       Itaguai 2018-03-13 18:36:14  973628819893641216   pt     false   

            state                                               text  
0  Rio de Janeiro  Hoje que tomei coragem de dar vacina da febre ...  
1       Sao Paulo  Faleceu um parente de uma amiga vítima de febr...  
2          Parana  Kakakaka tomei a vacina de febre amarela, e ag...  
3  Rio de Janeiro      Finalmente tomei a vacina da febre amarela 🙏💉  
4  Rio de Janeiro  Tomei a vacina contra a febre amarela, agora é...

In [431]:
tweetscoords = dfcoord.text
list(tweetscoords[:8])

['Hoje que tomei coragem de dar vacina da febre amarela a Mikaelly.',
 'Faleceu um parente de uma amiga vítima de febre amarela, a situação tá punk. Quem ainda não se vacinou, corra para… https://t.co/gm7Zo4vw3Y',
 'Kakakaka tomei a vacina de febre amarela, e agora to com febre haauahauahaua',
 'Finalmente tomei a vacina da febre amarela 🙏💉',
 'Tomei a vacina contra a febre amarela, agora é só orar pra não acontecer nada',
 'eu vou morrer se eu fizer a trilha da Prainha esse fds sem ter tomado a vacina da febre amarela?',
 'Aí você toma a vacina da febre amarela pra não morrer, e começa a ter 375 reações à vacina!',
 'preciso ir tomar a vacina da febre amarela']

In [432]:
# elimina url
def elimina_url(texto):
    #print(type(texto))
    texto = str(texto)
    return re.sub(r'http\S+', '', texto)


tweetscoords2 = tweetscoords.copy()
for i in range(len(tweetscoords2)):
    tweetscoords2[i] = elimina_url(tweetscoords2[i])
list(tweetscoords2[:8])

['Hoje que tomei coragem de dar vacina da febre amarela a Mikaelly.',
 'Faleceu um parente de uma amiga vítima de febre amarela, a situação tá punk. Quem ainda não se vacinou, corra para… ',
 'Kakakaka tomei a vacina de febre amarela, e agora to com febre haauahauahaua',
 'Finalmente tomei a vacina da febre amarela 🙏💉',
 'Tomei a vacina contra a febre amarela, agora é só orar pra não acontecer nada',
 'eu vou morrer se eu fizer a trilha da Prainha esse fds sem ter tomado a vacina da febre amarela?',
 'Aí você toma a vacina da febre amarela pra não morrer, e começa a ter 375 reações à vacina!',
 'preciso ir tomar a vacina da febre amarela']

In [433]:
print(tweetscoords2[1])

Faleceu um parente de uma amiga vítima de febre amarela, a situação tá punk. Quem ainda não se vacinou, corra para… 


In [434]:
freq_testes = vectorizer.transform(tweetscoords2)

In [435]:
freq_testes.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [436]:
# Fazendo a classificação com o modelo treinado.
modelo.predict(freq_testes)[:5]

array(['prevenção', 'notícia', 'prevenção', 'prevenção', 'prevenção'],
      dtype='<U9')

In [437]:
#tweets = ['text']
#classes = dataset2['classificação']

In [438]:
#type(tweets)

In [439]:
df = pd.DataFrame()
df['tweet'] = tweetscoords2
df['classificacao'] = modelo.predict(freq_testes)

tweets = tweetscoords2
classificacao = modelo.predict(freq_testes)

print ("Novas Instancias Classificadas...")
for i in range(len(df)):
    print (classificacao[i], " -> ", tweets[i] )

Novas Instancias Classificadas...
prevenção  ->  Hoje que tomei coragem de dar vacina da febre amarela a Mikaelly.
notícia  ->  Faleceu um parente de uma amiga vítima de febre amarela, a situação tá punk. Quem ainda não se vacinou, corra para… 
prevenção  ->  Kakakaka tomei a vacina de febre amarela, e agora to com febre haauahauahaua
prevenção  ->  Finalmente tomei a vacina da febre amarela 🙏💉
prevenção  ->  Tomei a vacina contra a febre amarela, agora é só orar pra não acontecer nada
outros  ->  eu vou morrer se eu fizer a trilha da Prainha esse fds sem ter tomado a vacina da febre amarela?
outros  ->  Aí você toma a vacina da febre amarela pra não morrer, e começa a ter 375 reações à vacina!
outros  ->  preciso ir tomar a vacina da febre amarela
prevenção  ->  Crl fui tomar vacina de febre amarela,e a mulher disse q eu quebrei a agulha na hora de injetar p causa do meu músc… 
outros  ->  MEU DEUS DO CÉU QUEM EU VOU TER QUE MATAR PRA PODER CONSEGUIR UMA VACINA DE FEBRE AMARELA?????
o

outros  ->  tá, mas e a febre amarela ????? cade
outros  ->  @pandlrcom Daqui a pouco é infectado pela febre amarela
outros  ->  achei interessante e fofo o frasco da vacina contra febre amarela ter uma tampinha amarela
prevenção  ->  Fui tomar vacina contra febre amarela e ouvi as pessoas no posto de saúde dizendo : Esse mosquito Aedes aegypti, tr… 
notícia  ->  Lauro Jardim hj no Globo noticia como coisa + normal do mundo deslocamento d 22 milhões destinados à campanhas d ut… 
prevenção  ->  Meu pai falo q era p eu ir c ele tomar a vacina da febre amarela. Aí falei ok né. Aí o engraçadinho fez eu tomar e… 
prevenção  ->  Queria dar uma dica para quem precisa viajar para um local que precisa da carteira internacional da vacina da febre… 
outros  ->  será q o harry tomou vacina de febre amarela?
outros  ->  @jorgecn Kkkkkk febre amarela
notícia  ->  Galerinha, terá vacinação na Linha 4-Amarela de metrô, em parceria c/ a Secretaria Municipal de Saúde, oferece aos… 
outros  ->  @farkalia

In [440]:
df.head(8)

tweet classificacao
0  Hoje que tomei coragem de dar vacina da febre ...     prevenção
1  Faleceu um parente de uma amiga vítima de febr...       notícia
2  Kakakaka tomei a vacina de febre amarela, e ag...     prevenção
3      Finalmente tomei a vacina da febre amarela 🙏💉     prevenção
4  Tomei a vacina contra a febre amarela, agora é...     prevenção
5  eu vou morrer se eu fizer a trilha da Prainha ...        outros
6  Aí você toma a vacina da febre amarela pra não...        outros
7         preciso ir tomar a vacina da febre amarela        outros

In [441]:
df.classificacao.value_counts()

outros       349
prevenção    309
notícia       84
doença        39
Name: classificacao, dtype: int64

In [442]:
dfcoord['classificacao'] = modelo.predict(freq_testes)
dfcoord.head()

city                data              id_str lang retweeted  \
0     Nilopolis 2018-03-13 20:27:36  973656844601315328   pt     false   
1  Praia Grande 2018-03-13 19:57:57  973649386491260929   pt     false   
2     Guaratuba 2018-03-13 19:53:57  973648377043922947   pt     false   
3       Itaguai 2018-03-13 19:16:13  973638884155326464   pt     false   
4       Itaguai 2018-03-13 18:36:14  973628819893641216   pt     false   

            state                                               text  \
0  Rio de Janeiro  Hoje que tomei coragem de dar vacina da febre ...   
1       Sao Paulo  Faleceu um parente de uma amiga vítima de febr...   
2          Parana  Kakakaka tomei a vacina de febre amarela, e ag...   
3  Rio de Janeiro      Finalmente tomei a vacina da febre amarela 🙏💉   
4  Rio de Janeiro  Tomei a vacina contra a febre amarela, agora é...   

  classificacao  
0     prevenção  
1       notícia  
2     prevenção  
3     prevenção  
4     prevenção

In [443]:
len(dfcoord)

781

In [444]:
dfcoord3 = dfcoord[dfcoord.classificacao != 'outros']
dfcoord3

city                data               id_str lang  \
0                     Nilopolis 2018-03-13 20:27:36   973656844601315328   pt   
1                  Praia Grande 2018-03-13 19:57:57   973649386491260929   pt   
2                     Guaratuba 2018-03-13 19:53:57   973648377043922947   pt   
3                       Itaguai 2018-03-13 19:16:13   973638884155326464   pt   
4                       Itaguai 2018-03-13 18:36:14   973628819893641216   pt   
8                       Itaguai 2018-03-13 17:30:31   973612281799106561   pt   
13                   Embu Guacu 2018-03-14 11:42:29   973887082677657602   pt   
17   Governador Dix Sept Rosado 2018-03-14 10:24:41   973867503750348800   pt   
20                   Imperatriz 2018-03-14 03:02:24   973756201224888321   pt   
24                   Embu Guacu 2018-03-15 22:04:08   974405916261888001   pt   
25                   Embu Guacu 2018-03-15 20:43:16   974385563942957061   pt   
27                   Embu Guacu 2018-03-16 20:42:40   974747803598704640   pt   
29               Artur Nogueira 2018-03-16 19:53:08   974735335799312385   pt   
30                     Luziania 2018-03-17 11:45:04   974974897733668865   pt   
32                   Embu Guacu 2018-03-19 15:25:38   975755181198626818   pt   
33                 Santa Isabel 2018-03-19 13:35:11   975727386045140992   pt   
36                  Pirapozinho 2018-03-19 04:19:01   975587420480602112   pt   
42                   Embu Guacu 2018-03-19 20:37:56   975833773345525760   pt   
45        Sao Lourenco da Serra 2018-03-19 18:39:07   975803871401521159   pt   
46                  Nova Iguacu 2018-03-19 18:16:29   975798175339565057   pt   
49                  Sao Fidelis 2018-03-20 00:39:35   975894585892392961   pt   
56                       Paraty 2018-03-20 17:57:45   976155850573303808   pt   
57                       Paraty 2018-03-20 17:08:43   976143510020808705   pt   
58                      Itaguai 2018-03-20 16:58:12   976140864396386304   pt   
60    Santo Antonio do Leverger 2018-03-20 21:48:24   976213897416933377   pt   
62                   Embu Guacu 2018-03-21 10:12:28   976401144330293248   pt   
63               Paraiba do Sul 2018-03-21 08:04:22   976368909430476800   pt   
65                   Embu Guacu 2018-03-21 04:10:39   976310092734836736   pt   
67                      Itaguai 2018-03-21 23:28:06   976601371624529921   pt   
70        Sao Bernardo do Campo 2018-03-22 02:49:37   976652085168099328   pt   
..                          ...                 ...                  ...  ...   
736              Paraiba do Sul 2018-06-24 12:47:06  1010866906126082049   pt   
737              Paraiba do Sul 2018-06-24 12:46:27  1010866742783086593   pt   
738              Paraiba do Sul 2018-06-24 12:46:03  1010866644648898560   pt   
739              Paraiba do Sul 2018-06-24 12:45:55  1010866611727814657   pt   
740              Paraiba do Sul 2018-06-23 23:40:00  1010668829096202240   pt   
742                Pires do Rio 2018-06-26 16:12:49  1011643453401370627   pt   
743                     Itaguai 2018-06-26 10:43:32  1011560585010515968   pt   
744              Paraiba do Sul 2018-06-26 10:07:11  1011551438907101186   pt   
745              Paraiba do Sul 2018-06-26 10:06:47  1011551336943489024   pt   
746              Paraiba do Sul 2018-06-26 10:02:53  1011550356499828736   pt   
747              Paraiba do Sul 2018-06-26 10:02:15  1011550195895668737   pt   
750              Rio Piracicaba 2018-06-28 02:25:19  1012159982203260928   pt   
751                Constitucion 2018-06-27 11:34:15  1011935739502825472   pt   
753                  Embu Guacu 2018-06-30 18:02:05  1013120501609197568   pt   
757                    Mongagua 2018-06-30 11:51:20  1013027199941738496   pt   
759               Caraguatatuba 2018-06-30 01:26:31  1012869962464137216   pt   
761                 Farroupilha 2018-06-29 02:14:18  1012519597495463936   pt   
764              Paraiba do Sul 2018-07-02 11:07:28  10137

In [445]:
len(dfcoord3)

432

** Avaliando o modelo **

In [446]:
len(classes)

4009

In [447]:
freq_tweets = vectorizer.transform(dfcoord3.classificacao)

In [448]:
# Fazendo o cross validation do modelo
classes = modelo.predict(freq_testes)
resultados = cross_val_predict(modelo, freq_tweets, list(dfcoord3.classificacao), cv=10)
resultados[:5]

array(['prevenção', 'notícia', 'prevenção', 'prevenção', 'prevenção'],
      dtype='<U9')

In [449]:
resultados[:10]

array(['prevenção', 'notícia', 'prevenção', 'prevenção', 'prevenção',
       'prevenção', 'notícia', 'notícia', 'prevenção', 'notícia'],
      dtype='<U9')

# comparacao entre modelos com e sem coordenadas

In [450]:
dfcoord3.head()

city                data              id_str lang retweeted  \
0     Nilopolis 2018-03-13 20:27:36  973656844601315328   pt     false   
1  Praia Grande 2018-03-13 19:57:57  973649386491260929   pt     false   
2     Guaratuba 2018-03-13 19:53:57  973648377043922947   pt     false   
3       Itaguai 2018-03-13 19:16:13  973638884155326464   pt     false   
4       Itaguai 2018-03-13 18:36:14  973628819893641216   pt     false   

            state                                               text  \
0  Rio de Janeiro  Hoje que tomei coragem de dar vacina da febre ...   
1       Sao Paulo  Faleceu um parente de uma amiga vítima de febr...   
2          Parana  Kakakaka tomei a vacina de febre amarela, e ag...   
3  Rio de Janeiro      Finalmente tomei a vacina da febre amarela 🙏💉   
4  Rio de Janeiro  Tomei a vacina contra a febre amarela, agora é...   

  classificacao  
0     prevenção  
1       notícia  
2     prevenção  
3     prevenção  
4     prevenção

In [451]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(dfcoord3.text)
modelo2 = MultinomialNB()
modelo2.fit(freq_tweets, list(dfcoord3.classificacao))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [452]:
# Fazendo o cross validation do modelo
classes = modelo2.predict(freq_tweets)
resultados = cross_val_predict(modelo2, freq_tweets, list(dfcoord3.classificacao), cv=10)

In [453]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(list(classes),resultados)

0.92592592592592593

In [454]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['REAL'], colnames=['PREDITO'], margins=True))

PREDITO    doença  notícia  prevenção  All
REAL                                      
doença         33        6          0   39
notícia         8       76          3   87
prevenção       2       13        291  306
All            43       95        294  432


### Matriz de Confusão

In [455]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(list(classes),resultados)

0.92592592592592593

In [456]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['REAL'], colnames=['PREDITO'], margins=True))

PREDITO    doença  notícia  prevenção  All
REAL                                      
doença         33        6          0   39
notícia         8       76          3   87
prevenção       2       13        291  306
All            43       95        294  432


In [457]:
# Medidas de validação do modelo
sentimento=['doença',  'notícia',    'prevenção' ]
print (metrics.classification_report(list(classes),resultados,sentimento))

             precision    recall  f1-score   support

     doença       0.77      0.85      0.80        39
    notícia       0.80      0.87      0.84        87
  prevenção       0.99      0.95      0.97       306

avg / total       0.93      0.93      0.93       432



# Usar o tweets com coordenadas classificados

In [458]:
dfcoord3.head()

city                data              id_str lang retweeted  \
0     Nilopolis 2018-03-13 20:27:36  973656844601315328   pt     false   
1  Praia Grande 2018-03-13 19:57:57  973649386491260929   pt     false   
2     Guaratuba 2018-03-13 19:53:57  973648377043922947   pt     false   
3       Itaguai 2018-03-13 19:16:13  973638884155326464   pt     false   
4       Itaguai 2018-03-13 18:36:14  973628819893641216   pt     false   

            state                                               text  \
0  Rio de Janeiro  Hoje que tomei coragem de dar vacina da febre ...   
1       Sao Paulo  Faleceu um parente de uma amiga vítima de febr...   
2          Parana  Kakakaka tomei a vacina de febre amarela, e ag...   
3  Rio de Janeiro      Finalmente tomei a vacina da febre amarela 🙏💉   
4  Rio de Janeiro  Tomei a vacina contra a febre amarela, agora é...   

  classificacao  
0     prevenção  
1       notícia  
2     prevenção  
3     prevenção  
4     prevenção

In [459]:
len(dfcoord3)

432

In [460]:
dfcoord3.state.value_counts()

Rio de Janeiro         247
Sao Paulo              124
Goias                   11
Rio Grande do Sul        9
Parana                   7
Minas Gerais             5
Mato Grosso              4
Rio Grande do Norte      4
Santa Catarina           3
Para                     3
Pernambuco               3
Bahia                    2
Paraiba                  2
Amazonas                 2
Maule                    1
Roraima                  1
Espirito Santo           1
Maranhao                 1
Tocantins                1
Alagoas                  1
Name: state, dtype: int64

In [461]:
#i = 0
#lin = dfcoord3.iloc[i,:]
#print(list(lin))
#print(lin.city, lin.state)

In [462]:
#dict = {}
#for i in range(len(dfcoord3)): 
##    lin = dfcoord3.iloc[i,:]
#    cidade = 0
#    estado = 5
#    print(lin[cidade])
#    dict[lin[cidade]] = lin[estado]

In [463]:
#len(dict)

In [464]:
dfcoord3.city.value_counts()

Paraiba do Sul                180
Embu Guacu                     61
Itaguai                        48
Caraguatatuba                  10
Sao Jose dos Campos             6
Luziania                        6
Nilopolis                       5
Ibirite                         4
Guaiba                          4
Praia Grande                    4
Guapo                           4
Sao Bernardo do Campo           4
Guarulhos                       3
Niteroi                         3
Sao Paulo                       3
Itatiaia                        3
Mongagua                        3
Embu                            3
Santo Antonio do Leverger       2
Sao Lourenco da Serra           2
Valparaiso                      2
Paraty                          2
Rio de Janeiro                  2
Indaiatuba                      2
Guaratuba                       2
Charqueada                      2
Itaparica                       2
Parnamirim                      2
Juquitiba                       2
Encruzilhada d

In [465]:
type(dfcoord3.city.value_counts())

pandas.core.series.Series

In [466]:
ocorrencia_cidades = dfcoord3.city.value_counts()
ocorrencia_cidades.head()

Paraiba do Sul         180
Embu Guacu              61
Itaguai                 48
Caraguatatuba           10
Sao Jose dos Campos      6
Name: city, dtype: int64

In [467]:
df = ocorrencia_cidades.to_frame().reset_index()
df = df.rename(columns={'index': 'cidade', 'city': 'ocorrencias'})
df.head()

cidade  ocorrencias
0       Paraiba do Sul          180
1           Embu Guacu           61
2              Itaguai           48
3        Caraguatatuba           10
4  Sao Jose dos Campos            6

In [489]:
df_cidade = df.to_excel('cidade_febre_amarela.xlsx', engine='xlsxwriter')



In [490]:
total_por_estado = dfcoord3.state.value_counts()
total_por_estado.head()

Rio de Janeiro       247
Sao Paulo            124
Goias                 11
Rio Grande do Sul      9
Parana                 7
Name: state, dtype: int64

In [491]:
dfestado = total_por_estado.to_frame().reset_index()
dfestado = dfestado.rename(columns={'index': 'estado', 'state': 'ocorrencias'})
dfestado.head(8)

estado  ocorrencias
0       Rio de Janeiro          247
1            Sao Paulo          124
2                Goias           11
3    Rio Grande do Sul            9
4               Parana            7
5         Minas Gerais            5
6          Mato Grosso            4
7  Rio Grande do Norte            4

In [492]:
df_estado = dfestado.to_excel('estado_febre_amarela.xlsx', engine='xlsxwriter')

In [493]:
len(df)

17

In [ ]:
#df['Value'] = df.apply(lambda row: my_test(row['a'], row['c']), axis=1)

In [496]:
df.estado.value_counts()

Bahia                  1
Pernambuco             1
Rio Grande do Sul      1
Minas Gerais           1
Paraiba                1
Maranhao               1
Parana                 1
Espirito Santo         1
Tocantins              1
Maule                  1
Alagoas                1
Para                   1
Rio de Janeiro         1
Goias                  1
Sao Paulo              1
Mato Grosso            1
Rio Grande do Norte    1
Name: estado, dtype: int64